In [24]:
pip install pandas requests mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import requests
import mysql.connector

print("All libraries imported successfully")

All libraries imported successfully


In [26]:
movies_df = pd.read_csv("../data/movies.csv")

In [9]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [21]:
ratings_df = pd.read_csv("../data/ratings.csv")

In [12]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [14]:
print("Movies rows:", movies_df.shape[0])
print("Ratings rows:", ratings_df.shape[0])

Movies rows: 9742
Ratings rows: 100836


In [15]:
print("Missing movieId in movies:", movies_df['movieId'].isna().sum())
print("Missing movieId in ratings:", ratings_df['movieId'].isna().sum())

Missing movieId in movies: 0
Missing movieId in ratings: 0


In [16]:
ratings_df['rating'].describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [17]:
invalid_ratings = ratings_df[~ratings_df['movieId'].isin(movies_df['movieId'])]
print("Ratings with invalid movieId:", invalid_ratings.shape[0])

Ratings with invalid movieId: 0


In [27]:
movies_df['release_year'] = movies_df['title'].str.extract(r'\((\d{4})\)')

In [28]:
movies_df['release_year'] = movies_df['release_year'].astype('Int64')

In [35]:
movies_df[['title', 'release_year']].head(10)

,title,release_year
0,Toy Story (1995),1995
1,Jumanji (1995),1995
2,Grumpier Old Men (1995),1995
3,Waiting to Exhale (1995),1995
4,Father of the Bride Part II (1995),1995
5,Heat (1995),1995
6,Sabrina (1995),1995
7,Tom and Huck (1995),1995
8,Sudden Death (1995),1995
9,GoldenEye (1995),1995


In [36]:
movies_df['release_year'].isna().sum()

np.int64(13)

In [1]:
with open("../omdb_api_key.txt", "r") as file:
    OMDB_API_KEY = file.read().strip()

print("API key loaded successfully")

API key loaded successfully


In [2]:
len(OMDB_API_KEY) 

8

In [3]:
import requests

test_title = "Toy Story"

url = "http://www.omdbapi.com/"
params = {
    "t": test_title,
    "apikey": OMDB_API_KEY
}

response = requests.get(url, params=params)
movie_data = response.json()

movie_data

{'Title': 'Toy Story',
 'Year': '1995',
 'Rated': 'G',
 'Released': '22 Nov 1995',
 'Runtime': '81 min',
 'Genre': 'Animation, Adventure, Comedy',
 'Director': 'John Lasseter',
 'Writer': 'John Lasseter, Pete Docter, Andrew Stanton',
 'Actors': 'Tom Hanks, Tim Allen, Don Rickles',
 'Plot': "A cowboy doll is profoundly jealous when a new spaceman action figure supplants him as the top toy in a boy's bedroom. When circumstances separate them from their owner, the duo have to put aside their differences to return to him.",
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 3 Oscars. 29 wins & 24 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BZTA3OWVjOWItNjE1NS00NzZiLWE1MjgtZDZhMWI1ZTlkNzYwXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '100%'},
  {'Source': 'Metacritic', 'Value': '96/100'}],
 'Metascore': '96',
 'imdbRating': '8.3',
 'imdbVotes'

In [4]:
def fetch_omdb_details(title, api_key):
    url = "http://www.omdbapi.com/"
    params = {
        "t": title,
        "apikey": api_key
    }
    
    try:
        response = requests.get(url, params=params, timeout=5)
        data = response.json()
        
        if data.get("Response") == "True":
            director = data.get("Director")
            
            runtime = data.get("Runtime")
            if runtime and runtime != "N/A":
                runtime_minutes = int(runtime.split()[0])
            else:
                runtime_minutes = None
            
            imdb_rating = data.get("imdbRating")
            if imdb_rating and imdb_rating != "N/A":
                imdb_rating = float(imdb_rating)
            else:
                imdb_rating = None
            
            return director, runtime_minutes, imdb_rating
        
        else:
            return None, None, None
    
    except Exception as e:
        return None, None, None

In [5]:
fetch_omdb_details("Toy Story", OMDB_API_KEY) # TEST CODE

('John Lasseter', 81, 8.3)

In [11]:
movies_df['director'] = None
movies_df['runtime_minutes'] = None
movies_df['imdb_rating'] = None

In [12]:
sample_movies = movies_df.head(10)
sample_movies[['title']]

,title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
5,Heat (1995)
6,Sabrina (1995)
7,Tom and Huck (1995)
8,Sudden Death (1995)
9,GoldenEye (1995)


In [13]:
for idx, row in sample_movies.iterrows():
    director, runtime, imdb_rating = fetch_omdb_details(row['title'], OMDB_API_KEY)
    
    movies_df.at[idx, 'director'] = director
    movies_df.at[idx, 'runtime_minutes'] = runtime
    movies_df.at[idx, 'imdb_rating'] = imdb_rating

In [14]:
movies_df.head(10)[['title', 'director', 'runtime_minutes', 'imdb_rating']]

,title,director,runtime_minutes,imdb_rating
0,Toy Story (1995),N/A,None,None
1,Jumanji (1995),N/A,None,None
2,Grumpier Old Men (1995),None,None,None
3,Waiting to Exhale (1995),None,None,None
4,Father of the Bride Part II (1995),None,None,None
5,Heat (1995),N/A,None,None
6,Sabrina (1995),None,None,None
7,Tom and Huck (1995),None,None,None
8,Sudden Death (1995),None,None,None
9,GoldenEye (1995),None,None,None


In [15]:
import time

In [17]:
enriched_count = 0

for idx, row in movies_df.iterrows():
    if enriched_count >= 300:
        break
    
    if movies_df.at[idx, 'director'] is None:
        director, runtime, imdb_rating = fetch_omdb_details(row['title'], OMDB_API_KEY)
        
        movies_df.at[idx, 'director'] = director
        movies_df.at[idx, 'runtime_minutes'] = runtime
        movies_df.at[idx, 'imdb_rating'] = imdb_rating
        
        enriched_count += 1
        time.sleep(0.2)  # avoid hitting API too fast

print("Total movies enriched:", enriched_count)

Total movies enriched: 300


In [18]:
movies_df[['director', 'runtime_minutes', 'imdb_rating']].notna().sum()

director           33
runtime_minutes     0
imdb_rating         0
dtype: int64

In [22]:
movies_db_df = movies_df.rename(columns={"movieId": "movie_id"})

ratings_db_df = ratings_df.rename(columns={
    "userId": "user_id",
    "movieId": "movie_id"
})

In [31]:
movies_db_df = movies_df.rename(columns={"movieId": "movie_id"})

In [37]:
print(movies_df.columns.tolist())

['movieId', 'title', 'genres', 'release_year']


In [38]:
movies_df['director'] = movies_df.get('director')
movies_df['runtime_minutes'] = movies_df.get('runtime_minutes')
movies_df['imdb_rating'] = movies_df.get('imdb_rating')

In [39]:
movies_df[['director', 'runtime_minutes', 'imdb_rating']].head()

,director,runtime_minutes,imdb_rating
0,None,None,None
1,None,None,None
2,None,None,None
3,None,None,None
4,None,None,None


In [40]:
movies_db_df = movies_df.rename(columns={"movieId": "movie_id"})

movies_db_df = movies_db_df[
    ["movie_id", "title", "release_year", "genres", "director", "runtime_minutes", "imdb_rating"]
]

In [51]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [52]:
import mysql.connector
print("MySQL connector installed successfully!")

MySQL connector installed successfully!


In [60]:
import mysql.connector

# Replace the placeholders with your actual database credentials
conn = mysql.connector.connect(
    host="localhost",      # or your MySQL server IP
    user="root",
    password="Amitha@2003",
    database="movie_pipeline"
)

# Check if connection was successful
if conn.is_connected():
    print("Connected to MySQL database successfully!")

# Create a cursor object to execute queries
cursor = conn.cursor()

Connected to MySQL database successfully!


In [54]:
# Example: Create a table
create_table_query = """
CREATE TABLE IF NOT EXISTS students (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    age INT
)
"""
cursor.execute(create_table_query)
print("Table created successfully!")

# Example: Insert data
insert_query = "INSERT INTO students (name, age) VALUES (%s, %s)"
data = ("Junu", 22)
cursor.execute(insert_query, data)
conn.commit()  # Commit changes to the database
print("Data inserted successfully!")

# Example: Retrieve data
select_query = "SELECT * FROM students"
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
    print(row)

Table created successfully!
Data inserted successfully!
(1, 'Junu', 22)
(2, 'Junu', 22)


In [58]:
movies_db_df = movies_db_df.astype(object).where(pd.notnull(movies_db_df), None)
ratings_db_df = ratings_db_df.astype(object).where(pd.notnull(ratings_db_df), None)

In [67]:
import mysql.connector
from mysql.connector import Error

connection = mysql.connector.connect(
    host="localhost",      # or your MySQL server IP
    user="root",
    password="Amitha@2003",
    database="movie_pipeline"
)

cursor = connection.cursor()
print("Reconnected to MySQL")

Reconnected to MySQL


In [69]:
cursor = connection.cursor()

In [70]:
movies_data = movies_db_df.head(500).values.tolist()
cursor.executemany(insert_movies_query, movies_data)
connection.commit()

print("Movies data inserted successfully")

Movies data inserted successfully


In [71]:
cursor.execute("SELECT COUNT(*) FROM movies;")
cursor.fetchall()

[(500,)]

In [81]:
valid_movie_ids = set(movies_db_df.head(500)["movie_id"])

filtered_ratings_df = ratings_db_df[ratings_db_df["movie_id"].isin(valid_movie_ids)]

In [82]:
insert_ratings_query = """
INSERT INTO ratings (
    user_id,
    movie_id,
    rating,
    rating_timestamp
)
VALUES (%s, %s, %s, %s)
"""

In [79]:
ratings_db_df = ratings_db_df.rename(columns={"timestamp": "rating_timestamp"})

In [83]:
ratings_data = filtered_ratings_df.values.tolist()
cursor.executemany(insert_ratings_query, ratings_data)
connection.commit()

print("Ratings data inserted successfully")

Ratings data inserted successfully


In [84]:
cursor.execute("SHOW TABLES;")
cursor.fetchall()

[('movies',), ('ratings',), ('students',)]

In [ ]:
try:
    # Your database operations here
    pass
finally:
    cursor.close()
    conn.close()
    print("Connection closed in finally block.")
